In [2]:
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.models import WordLevel, BPE
from tokenizers.pre_tokenizers import Whitespace,Split,ByteLevel, WhitespaceSplit
from tokenizers.normalizers import Lowercase, NFKC
import os
import polars as pl
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
from tqdm import tqdm
import time
import json
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordLevelTrainer, WordPieceTrainer
import gc
from transformers import AutoConfig, AutoTokenizer, AutoModel, DataCollatorWithPadding
import datasets
from datasets import Dataset

multiprocessing.cpu_count()

80

In [2]:
train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv').select(
        pl.col('molecule').alias('smiles'),
#         pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('BRD4', 'HSA', 'sEH').cast(pl.Boolean),
    ).collect()
print(train_df.estimated_size('gb'), 'GB')
train_df

6.876520295627415 GB


smiles,BRD4,HSA,sEH
str,bool,bool,bool
"""C#CCOc1ccc(CNc…",false,false,false
"""C#CCOc1ccc(CNc…",false,false,false
"""C#CCOc1ccc(CNc…",false,false,false
"""C#CCOc1ccc(CNc…",false,false,false
"""C#CCOc1ccc(CNc…",false,false,false
…,…,…,…
"""[N-]=[N+]=NCCC…",false,false,false
"""[N-]=[N+]=NCCC…",false,false,false
"""[N-]=[N+]=NCCC…",false,false,false


In [3]:
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v4.csv').select(
        pl.col('molecule').alias('smiles'),
#         pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        # pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
    ).collect()
print(test_df.estimated_size('gb'), 'GB')
test_df

0.06128192972391844 GB


smiles
str
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
…
"""Cn1ncc2cc(Nc3n…"
"""[N-]=[N+]=NCCC…"
"""COC(=O)c1ccnc(…"


In [15]:
# !rm -r /home/dangnh36/datasets/competitions/leash_belka/processed/hf/

In [16]:
dataset = Dataset.from_polars(train_df)
dataset.save_to_disk(
        dataset_path = '/home/dangnh36/datasets/competitions/leash_belka/processed/hf/datasets/train',
        max_shard_size = None,
        num_shards = None,
        num_proc = None,
        storage_options = None,
)

Saving the dataset (0/17 shards):   0%|          | 0/98415610 [00:00<?, ? examples/s]

In [28]:
dataset = Dataset.from_polars(test_df)
dataset.save_to_disk(
        dataset_path = '/home/dangnh36/datasets/competitions/leash_belka/processed/hf/datasets/test',
        max_shard_size = None,
        num_shards = None,
        num_proc = None,
        storage_options = None,
)

Saving the dataset (0/1 shards):   0%|          | 0/878022 [00:00<?, ? examples/s]

In [3]:
from datasets import load_from_disk, concatenate_datasets

In [4]:
train_ds = load_from_disk('/home/dangnh36/datasets/competitions/leash_belka/processed/hf/datasets/train',
                          keep_in_memory=False)
len(train_ds)

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

98415610

In [5]:
test_ds = load_from_disk('/home/dangnh36/datasets/competitions/leash_belka/processed/hf/datasets/test',
                          keep_in_memory=False)
len(test_ds)

878022

In [6]:
ds = concatenate_datasets([train_ds, test_ds])
len(ds)

99293632

In [7]:
ds.save_to_disk(
        dataset_path = '/home/dangnh36/datasets/competitions/leash_belka/processed/hf/datasets/all',
        max_shard_size = None,
        num_shards = None,
        num_proc = None,
        storage_options = None,
)

Saving the dataset (0/17 shards):   0%|          | 0/99293632 [00:00<?, ? examples/s]